In [1]:
import xarray as xr
import psutil
import dask
from dask.distributed import Client

In [2]:
def setup_dask_client(
    workload_type="io",        # "cpu", "io", or "mixed"
    max_workers=None,           # Limit max workers (default = all cores)
    reserve_mem_gb=50,          # Memory reserved for system and overhead
    max_mem_gb=None,            # Cap usable memory if needed
    dashboard=True
):
    """
    Setup Dask client with auto-scaling for workload type.

    Parameters:
        workload_type (str): Type of workload - "cpu", "io", or "mixed"
        max_workers (int): Max logical cores to use (default = system max)
        reserve_mem_gb (int): System memory to reserve
        max_mem_gb (int): Cap memory usage (defaults to system memory)
        dashboard (bool): Whether to print the dashboard link

    Returns:
        dask.distributed.Client
    """
    assert workload_type in ("cpu", "io", "mixed"), "Invalid workload_type"

    logical_cores = psutil.cpu_count(logical=True)
    total_memory_gb = psutil.virtual_memory().total / 1e9

    if max_workers is None:
        max_workers = logical_cores

    if max_mem_gb is None:
        max_mem_gb = total_memory_gb

    usable_mem_gb = max_mem_gb - reserve_mem_gb

    # Recommended presets based on workload type
    if workload_type == "cpu":
        threads_per_worker = 1
        n_workers = min(max_workers, logical_cores)
    elif workload_type == "io":
        threads_per_worker = 8
        n_workers = max(1, logical_cores // threads_per_worker)
    else:  # "mixed"
        threads_per_worker = 4
        n_workers = max(1, logical_cores // threads_per_worker)

    memory_per_worker = usable_mem_gb // n_workers

    client = Client(
        n_workers=n_workers,
        threads_per_worker=threads_per_worker,
        memory_limit=f"{int(memory_per_worker)}GB"
    )

    if dashboard:
        print(f"Dask dashboard: {client.dashboard_link}")

    return client

In [3]:
client = setup_dask_client(workload_type="io")
client

Dask dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/8787/status


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/8787/status,
Dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/8787/status,Workers: 26
Total threads: 208,Total memory: 435.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43313,Workers: 26
Dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/8787/status,Total threads: 208
Started: Just now,Total memory: 435.86 GiB
Comm: tcp://127.0.0.1:34363,Total threads: 8
Dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/43863/status,Memory: 16.76 GiB
Nanny: tcp://127.0.0.1:38193,


In [4]:
path = '/g/data/hh5/tmp/pa1490/Global_HWs/Data/tas/Model_Data/CMIP6/ssp126/Global_Data/gridded_absolute_temperature/Global_gridded_absolute_temperature_CMIP6_CCCma_CanESM5_ssp126_r10i1p1f1_day_tas.nc'

In [5]:
ds = xr.open_mfdataset(path, 
                       parallel=True, 
                       chunks={'time': 100, 'lat': -1, 'lon': -1},
                       engine="h5netcdf"
                      )

In [6]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (time: 31390, bnds: 2, lon: 128, lat: 64)
Coordinates:
  * time       (time) object 251kB 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lon        (lon) float64 1kB 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
  * lat        (lat) float64 512B -87.86 -85.1 -82.31 ... 82.31 85.1 87.86
    height     float64 8B ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object 502kB dask.array<chunksize=(100, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 2kB dask.array<chunksize=(128, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 1kB dask.array<chunksize=(64, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 1GB dask.array<chunksize=(100, 64, 128), meta=np.ndarray>
Attributes: (12/55)
    CDI:                         Climate Data Interface version 2.1.0 (https:...
    source:                      CanESM5 (2019): \naerosol: interactive\natmo...
    institution:                 Canadian Centre for Climate Modelling and An...
    Conventions:                 CF-1.7 CMIP-6.2
    CCCma_model_hash:            24718c8346665b218729640ffe79d263b76456c4
    CCCma_parent_runid:          rc3.1-his10
    ...                          ...
    variable_id:                 tas
    variant_label:               r10i1p1f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.4.0
    CDO:                         Climate Data Operators version 2.1.0 (https:...

In [7]:
encod = {'zlib':True,'complevel':9, 'shuffle': True}
encoding1 = { v : encod for v in ds.data_vars}
encoding1['time'] = {}  # Let Xarray infer time units and dtype

write_task = ds.to_netcdf(
    '/g/data/gb02/sg7549/philipp/test_netcdf_z4.nc',
    encoding=encoding1,
    compute=True,
    engine="netcdf4"
)

/jobfs/145565013.gadi-pbs/ipykernel_463582/4184112288.py:5: UserWarning: Variable time has datetime type and a bounds variable but time.encoding does not have units specified. The units encodings for time and time_bnds will be determined independently and may not be equal, counter to CF-conventions. If this is a concern, specify a units encoding for time before writing to a file.
  write_task = ds.to_netcdf(
